In [6]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy as sp
import anndata
import gc
import matplotlib as mpl
import scipy
import loompy
from matplotlib import pyplot as plt

In [68]:
#ds = loompy.connect("D:\\Data\\stereo-seq\\l5_all.loom")
#st_adata=anndata.read_h5ad('D:\\Data\\stereo-seq\\SS200000135TL_D1_20.spatial.cluster.h5ad')
#marker_df = pd.read_excel('D:/Data/stereo-seq/mmc4.xlsx', sheet_name=None)


ds = loompy.connect("l5_all.loom")
st_adata=anndata.read_h5ad('SS200000135TL_D1_20.spatial.cluster.h5ad')
marker_df = pd.read_excel('mmc4.xlsx', sheet_name=None)



st_adata.layers["counts"] = st_adata.raw.X.copy()

sc.pp.highly_variable_genes(st_adata, layer="counts", flavor='seurat_v3',n_top_genes=5000)
st_gene_list = list(st_adata.var_names)

high_gene_lst = []
for i in range(len(st_adata.var['highly_variable'])):
    flag = list(st_adata.var['highly_variable'])[i]
    if flag is True:
        high_gene_lst.append(list(st_adata.var_names)[i])

        
marker_gene_lst = []
gene_marker_dict = {} 
for index, row in marker_df['markers_spec_selec_rob_10-May-2'].iterrows():
    if row['Unnamed: 1'] == 'Marker':
        ct = row['Cluster ID']
        marker_lst = [row['Gene1'], row['Gene2'], row['Gene3'], row['Gene4'], row['Gene5']]
        gene_marker_dict.update({ct : marker_lst})
        for gene in marker_lst:
            if gene not in marker_gene_lst and gene in st_gene_list:
                marker_gene_lst.append(gene)
        
        



In [69]:
len(filter_ct)

224

In [71]:
len(tmp_filter_cell_lst)

159402

In [70]:
from collections import Counter

count_ct_dict = Counter(ds.ca['ClusterName'])
filter_ct = []
for ct in gene_marker_dict:
    if ct not in ds.ca['ClusterName'] or count_ct_dict[ct] < 50:
        continue
    filter_ct.append(ct)


tmp_filter_cell_idx_lst = []
tmp_filter_cell_lst = []
for i in range(len(ds.ca['ClusterName'])):
    if ds.ca['ClusterName'][i] in filter_ct:
        tmp_filter_cell_idx_lst.append(i)
        tmp_filter_cell_lst.append(ds.ca['CellID'][i])


In [50]:
sc_gene_lst = ds.ra.Gene.tolist()

In [75]:
tmp_common_gene_lst = []
tmp_ls = []
for gene in high_gene_lst:
    if gene in sc_gene_lst:
        gene_idx = sc_gene_lst.index(gene)
        tmp_ls.append(ds[gene_idx, tmp_filter_cell_idx_lst])
        tmp_common_gene_lst.append(gene)

len(tmp_common_gene_lst)

4316

In [76]:
from scipy import sparse
expr_matrix = sparse.csr_matrix(np.array(tmp_ls).T)
adata = anndata.AnnData(expr_matrix)
adata.obs_names = tmp_filter_cell_lst
adata.var_names = tmp_common_gene_lst

tmp_sc_adata = adata

In [77]:
tmp_sc_adata.obs['cell_type'] = ds.ca['ClusterName'][tmp_filter_cell_idx_lst]

In [79]:
tmp_type_top_genes_dict ={}
sc.tl.rank_genes_groups(tmp_sc_adata, 'cell_type', method='wilcoxon')
for cell_type in filter_ct:
    tmp_type_top_genes_dict.update({cell_type:pd.DataFrame(tmp_sc_adata.uns['rank_genes_groups']['names'])[cell_type].tolist()})

... storing 'celltype' as categorical


C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:417: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme

C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'scores'] = scores[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:399: P

C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'scores'] = scores[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:399: P

C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:420: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'logfoldchanges'] = np.log2(
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
C:\Users\tutoradmin\AppData\Roaming\Python\Python38\site-packages\scanpy\tools\_rank_genes_groups.py:396: Perform

In [83]:
scanpy_marker_ls = []
for ct in tmp_type_top_genes_dict:
    if ct not in filter_ct:
        continue
    for gene in tmp_type_top_genes_dict[ct][:50]:
        if gene not in scanpy_marker_ls:
            scanpy_marker_ls.append(gene)

tmp_marker_gene_lst = []
gene_marker_dict = {} 
for index, row in marker_df['markers_spec_selec_rob_10-May-2'].iterrows():
    if row['Unnamed: 1'] == 'Marker':
        ct = row['Cluster ID']
        if ct not in filter_ct:
            continue
        marker_lst = [row['Gene1'], row['Gene2'], row['Gene3'], row['Gene4'], row['Gene5']]
        gene_marker_dict.update({ct : marker_lst})
        for gene in marker_lst:
            if gene not in tmp_marker_gene_lst and gene in st_gene_list:
                tmp_marker_gene_lst.append(gene)
            
            
marker_genes = []
for gene in st_gene_list:
    if gene in ds.ra['Gene']:
        if gene in tmp_marker_gene_lst or gene in scanpy_marker_ls:
            marker_genes.append(gene)
len(marker_genes)



1857